In [2]:
import numpy as np
# from scipy.signal import find_peaks, peak_prominences
import pyuff
import os                       # OS stuff
from tqdm import tqdm               #progressbar

In [38]:
def load_uff(): 
    ordnerpfad = "Data"

    if not os.path.exists(ordnerpfad):
        os.makedirs(ordnerpfad)
        os.makedirs(ordnerpfad+"/uff")
        os.makedirs(ordnerpfad+"/NumPyArrays")

        input("The Data directory didn't exist. I made it. Please save your uff files there. Press any key to continue")

    meas = ordnerpfad+'/'

    records=os.listdir(meas+'uff')

    print(str(len(records))+" files found in the Data Directory. Please type: \n")
    
    for i in range(len(records)): 
        print(str(i)+" for "+str(records[i]))

    decision = input()
    decision = int(decision)

    if decision > len(records)-1: 
       print('Error: No valid input.')

    mess=records[decision][:-4]

    path=meas+'uff/'+mess+".uff"
    uff_file = pyuff.UFF(path)

    # geometrie auslesen (type 15 bzw. 2411 (double) )
    IDs = np.where(uff_file.get_set_types() == 2411)

    if len(IDs[0])==1:
        Nodes=uff_file.read_sets(IDs[0][0])
    else:
        print("Error")

    # get recording date
    IDs = np.where(uff_file.get_set_types() == 151)
    uff_file.read_sets(IDs[0][0])['date_db_created']

    # Laser- und Hammer-signal suchen und speichern
    veloIDs=[]
    forceIDs=[]

    for i in tqdm(range(len(uff_file.get_set_types()))):
        if uff_file.get_set_types()[i]==58: #daten suchen
            if uff_file.read_sets(i)['id1']=='Response Time Trace': 
                if uff_file.read_sets(i)['id2']=='Vib  Geschwindigkeit': 
                    veloIDs.append(i)
            if uff_file.read_sets(i)['id1']=='Reference Time Trace':             
                if uff_file.read_sets(i)['id2']=='Ref1  Kraft': 
                    forceIDs.append(i)
            
    velocities = uff_file.read_sets(veloIDs)
    forces = uff_file.read_sets(forceIDs)

    LL=len(velocities)

    #messwerte und Positionen in NumPy-Array schreiben

    X=np.zeros(LL)
    Y=np.zeros(LL)
    Z=np.zeros(LL)

    Vel=np.zeros((LL,len(velocities[1]['x'])))
    Force=np.zeros((LL,len(velocities[1]['x'])))
    for i in range(LL):
        Force[i]=(forces[i]['data']) 
        Vel[i]=(velocities[i]['data']) 
        node_id=velocities[i]['rsp_node']
        id=np.where(Nodes['node_nums']==node_id)[0][0] 
        X[i]=Nodes['x'][id]
        Y[i]=Nodes['y'][id]
        Z[i]=Nodes['z'][id]

    np.save(meas + 'NumPyArrays/'+ mess + '_Xnodes', X)
    np.save(meas + 'NumPyArrays/'+ mess + '_Ynodes', Y)
    np.save(meas + 'NumPyArrays/'+ mess + '_Znodes', Z)
    np.save(meas + 'NumPyArrays/'+ mess + '_velocities', Vel)
    np.save(meas + 'NumPyArrays/'+ mess + '_forces', Force)


In [41]:
load_uff()

2 files found in the Data Directory. Please type: 

0 for Diatomischer Kristall Kurs SoSe 23.uff
1 for Diatomischer Kristall M2.uff
No valid input.


IndexError: list index out of range